In [1]:
from personal_twitter_data import TweetLoader
from mongo_data import MongoTweetStore
import pymongo

In [21]:
loader = TweetLoader()
loader.extract_tweets(count=10)

In [22]:
len(loader.get_loaded_tweets_as_json())

10

In [23]:
for tweet in loader.tweets:
    print(tweet.author.name)
    # print(tweet._json)

Bloomberg
Forbes
Bloomberg
Fox News
ESPN
Boston Celtics
Forbes
The Wall Street Journal
Fox News
Food Network


In [2]:
store = MongoTweetStore("twitter_dashboard_db", "home_timeline")

In [25]:
store.collection.create_index([("id", pymongo.ASCENDING)], name="tweet_id", unique=True, default_language="english")

'tweet_id'

In [26]:
store.save_tweets_to_db(loader.tweets)

1. FoodNetwork
2. WSJ
3. FoxNews
4. Forbes
5. celtics
6. espn
7. FoxNews
8. Forbes
9. business
10. business


In [28]:
data[0].keys()

dict_keys(['_id', 'created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'extended_entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'possibly_sensitive_appealable', 'lang'])

In [7]:
data = store.load_data_test(users=["business", "Forbes"], n_tweets=100, n_user_tweets=100)

business
9
Forbes
7


In [5]:
from mongo_data import MongoTweetStore
store = MongoTweetStore("twitter_dashboard_db", "home_timeline")

# Data is a list of dicts (user -> list(tweets))
u = ["business", "Forbes"]
# data = store.load_data_test(users=u, n_tweets=200, n_user_tweets=10, latest=False)
data = store.load_data_test(groupby_user=False)
[t["created_at"] for t in data["business"]]
data

# d = []
# for group in data:
#     d.append(group)
#     # print(type(group))
#     print(group.keys())

{'business': [{'_id': ObjectId('640625744968c3d1d15a8e33'),
   'created_at': 'Mon Mar 06 17:31:07 +0000 2023',
   'id': 1632795958634315776,
   'id_str': '1632795958634315776',
   'text': 'Home prices in the US declined for a sixth straight month, sending a key index of values down 2.7% from its peak in… https://t.co/sBzCDV59uM',
   'truncated': True,
   'entities': {'hashtags': [],
    'symbols': [],
    'user_mentions': [],
    'urls': [{'url': 'https://t.co/sBzCDV59uM',
      'expanded_url': 'https://twitter.com/i/web/status/1632795958634315776',
      'display_url': 'twitter.com/i/web/status/1…',
      'indices': [117, 140]}]},
   'source': '<a href="http://www.socialflow.com" rel="nofollow">SocialFlow</a>',
   'in_reply_to_status_id': None,
   'in_reply_to_status_id_str': None,
   'in_reply_to_user_id': None,
   'in_reply_to_user_id_str': None,
   'in_reply_to_screen_name': None,
   'user': {'id': 34713362,
    'id_str': '34713362',
    'name': 'Bloomberg',
    'screen_name': 'bus

In [9]:
type(d[0]["docs"][0])

IndexError: list index out of range

In [ ]:
data = store.load_data(latest=True)

for user_data in data:
    print(user_data["user"])
    print(len(user_data["tweets"]))
    for tweet in user_data["tweets"]:
        print(tweet["created_at"])

In [7]:
from pprint import pprint

def print_documents(cursor):
    for document in cursor:
        pprint(document)
        # print(document["user"]["screen_name"])
        # print(document["id"])
        # print(document["text"])

# store.collection.aggregate(["$match": {"$in": ["WSJ"]}])
data = store.collection.find({"user.screen_name": {"$in": ["WSJ", "business"]}})
print_documents(data)


{'_id': ObjectId('640625744968c3d1d15a8e33'),
 'contributors': None,
 'coordinates': None,
 'created_at': 'Mon Mar 06 17:31:07 +0000 2023',
 'entities': {'hashtags': [],
              'symbols': [],
              'urls': [{'display_url': 'twitter.com/i/web/status/1…',
                        'expanded_url': 'https://twitter.com/i/web/status/1632795958634315776',
                        'indices': [117, 140],
                        'url': 'https://t.co/sBzCDV59uM'}],
              'user_mentions': []},
 'favorite_count': 19,
 'favorited': False,
 'geo': None,
 'id': 1632795958634315776,
 'id_str': '1632795958634315776',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'place': None,
 'possibly_sensitive': False,
 'possibly_sensitive_appealable': False,
 'retweet_count': 8,
 'retweeted': False,
 'source': '<a href="http://www.soc

In [ ]:
users = ["WSJ", "business", "NASA"]
n_tweets = 1000
n_user_tweets = 1000
num = 2
latest = False
pipeline = [
    # Filter only requested users
    # {"$match": {"user.screen_name": {"$in": users}}},
    # Group tweets by user
    {
        "$group": {
            "_id": "$user.screen_name",
            "docs": {"$push": "$$ROOT"}
            },
    },
]

data = store.collection.aggregate(pipeline)
# print_documents(data)

# Order alphabetically by name (Mongo doesn't return in specific order)
data = sorted(data, key=lambda x: x["_id"].lower())

tweets_count = 0
for group in data:
    if tweets_count >= n_tweets:
        break

    tweets = group["docs"]
    print(group["_id"])

    if len(tweets) > n_tweets - tweets_count:
        tweets = tweets[: n_tweets - tweets_count]

    if n_user_tweets:
        tweets = tweets[:n_user_tweets]
        # print(group["docs"])
        # print(len(group["docs"]))
        print(len(tweets))

    tweets_count += len(tweets)